# Advent of Code 2019

___

[**Day 1**](#day1) &nbsp; &nbsp; &nbsp; [Day 2](#day2) &nbsp; &nbsp; &nbsp; [Day 3](#day3) &nbsp; &nbsp; &nbsp; [Day 4](#day4) &nbsp; &nbsp; &nbsp; [Day 5](#day5)

[Day 6](#day6) &nbsp; &nbsp; &nbsp; [Day 7](#day7) &nbsp; &nbsp; &nbsp; [Day 8](#day8) &nbsp; &nbsp; &nbsp; [Day 9](#day9) &nbsp; &nbsp; &nbsp; [Day 10](#day10)

[Day 11](#day11) &nbsp; &nbsp; [Day 12](#day12) &nbsp; &nbsp; [Day 13](#day13) &nbsp; &nbsp; [Day 14](#day14) &nbsp; &nbsp; [Day 15](#day15)

[Day 16](#day16) &nbsp; &nbsp; [Day 17](#day17) &nbsp; &nbsp; [Day 18](#day18) &nbsp; &nbsp; [Day 19](#day19) &nbsp; &nbsp; [Day 20](#day20)

[Day 21](#day21) &nbsp; &nbsp; [Day 22](#day22) &nbsp; &nbsp; [Day 23](#day23) &nbsp; &nbsp; [Day 24](#day24) &nbsp; &nbsp; [Day 25](#day25)

___

<a class="anchor" id="day1"></a>
# Day 1

*Part 1*  
Launching a bunch of modules into space, need to calculate how much fuel we need based on their masses. To find fuel needed for a module, divide mass by 3, round down, subtract 2 (negatives are 0). Find total fuel needed.

*Part 2*  
Fuel has weight too, and so we need more fuel based on the fuel we need.  Recursively apply the same pattern to Part 1 to find the actual total fuel we need. (Ex: module mass of 47 -> needs 13 fuel -> needs 2 fuel, total of 15 fuel for that module)

In [2]:
with open('data2019/day1.txt') as f1:
    masses = [int(row.strip()) for row in f1.readlines()]
    
masses[-5:]

[97162, 80664, 149742, 88983, 74518]

**Part 1**

In [3]:
fuels = [(mass // 3) - 2 for mass in masses]
total_fuel1 = sum(fuels)
total_fuel1

3289802

**Part 2**

In [4]:
def get_recursive_fuel(mass):
    fuel_req = 0
    fuel = (mass // 3) - 2
    fuel_req += fuel
    
    while fuel > 8:
        fuel = (fuel // 3) - 2
        fuel_req += fuel
    
    return fuel_req

get_recursive_fuel(100756)

50346

In [5]:
total_fuel2 = sum([get_recursive_fuel(mass) for mass in masses])
total_fuel2

4931831

<a class="anchor" id="day2"></a>

# Day 2

It's freakin intcode time.

If, at index i, there is a 1, then add the values at locations indicated by value i+1 and i+2 and store the sum in location i+3.
If, at index i, there is a 2, then multiply the values at locations indicated by value i+1 and i+2 and store the sum in location i+3.
If, at index i, there is a 99, then terminate the program.

*Part 1*  
Given the puzzle input, replace index 1 with value 12 and index 2 with the value 2, run the program, determine the value at index 0.

*Part 2*
Now, find the (a, b) pair with 0 <= a, b <= 100 that, when inserted at the 1st and 2nd indices as in Part 1, result in an output of 19690720 at index 0 after the program terminates. Report solution as 100*a + b.


Note - this depends on the *Intcode()* data class in the file *intcode.py*.

In [6]:
with open('data2019/day2.txt') as f2:
    program = [int(x) for x in f2.read().split(',')]

program[:5], program[-5:]

([1, 0, 0, 3, 1], [99, 2, 0, 14, 0])

**Part 1**

In [8]:
from data2019.intcode import Intcode

program1 = program[:]
program1[1] = 12
program1[2] = 2
computer = Intcode(program1)
computer.run_program()
computer.program[0]

4570637

**Part 2**

In [10]:
from data2019.intcode import Intcode

for a in range(100):
    for b in range(100):
        prog = program[:]
        prog[1] = a
        prog[2] = b
        computer = Intcode(prog)
        computer.run_program()
        output = computer.program[0]
        if output == 19690720:
            print(f'{a = } and {b = } give the desired output with a puzzle solution of {100*a + b}')

a = 54 and b = 85 give the desired output with a puzzle solution of 5485


<a class="anchor" id="day3"></a>
# Day 3

*Part 1*  
Given 2 wire diagrams, find the nearest intersection location from the central node (Manhattan measure).

*Part 2*  
d

In [12]:
with open('data2019/day3.txt') as f3:
    wire1, wire2 = [row.strip() for row in f3.readlines()]
    
wire1, wire2 = wire1.split(','), wire2.split(',')
wire1[-10:]

['D33', 'R831', 'D783', 'R577', 'U402', 'R482', 'D741', 'R737', 'D474', 'L666']

In [13]:
len(wire1), len(wire2)

(301, 301)

In [14]:
import numpy as np

samplewire1 = 'R98,U47,R26,D63,R33,U87,L62,D20,R33,U53,R51'.split(',')
samplewire2 = 'U98,R91,D20,R16,D67,R40,U7,R15,U6,R7'.split(',')
wire1 = samplewire1
wire2 = samplewire2

# build location lists for each wire
locs1, locs2 = [], []
w1, w2 = np.zeros(2), np.zeros(2)
dist1, dist2 = 0, 0
moves = {'U': np.array([0, 1]), 'D': np.array([0, -1]), 'L': np.array([-1, 0]), 'R': np.array([1, 0])}
dist_sums = []

for i in range(len(wire1)):
    inst1 = wire1[i]
    char, val = inst1[0], int(inst1[1:])
    for j in range(val):
        w1 += moves[char]
        dist1 += 1
        locs1.append((w1[0], w1[1], dist1))
        
for i in range(len(wire2)):
    inst2 = wire2[i]
    char, val = inst2[0], int(inst2[1:])
    for j in range(val):
        w2 += moves[char]
        dist2 += 1
        matches = [(w2[0], w2[1]) == (loc[0], loc[1]) for loc in locs1]
        if any(matches):
            idx1 = matches.index(True)
            partial_dist1 = locs1[idx1][2]
            dist_sums.append(partial_dist1 + dist2)
        #locs2.append(tuple(w2))

min(dist_sums)

410

In [15]:
# find set intersections
intersections = set(locs1).intersection(set(locs2))
intersections

set()

In [17]:
#min([abs(x) + abs(y) for (x, y) in intersections])

<a class="anchor" id="day4"></a>

# Day 4

<a class="anchor" id="day5"></a>

# Day 5

<a class="anchor" id="day6"></a>

# Day 6

<a class="anchor" id="day7"></a>

# Day 7

<a class="anchor" id="day8"></a>

# Day 8

<a class="anchor" id="day9"></a>

# Day 9

<a class="anchor" id="day10"></a>

# Day 10

<a class="anchor" id="day11"></a>

# Day 11

<a class="anchor" id="day12"></a>

# Day 12

<a class="anchor" id="day13"></a>

# Day 13

<a class="anchor" id="day14"></a>

# Day 14

<a class="anchor" id="day15"></a>

# Day 15

<a class="anchor" id="day16"></a>

# Day 16

<a class="anchor" id="day17"></a>

# Day 17

<a class="anchor" id="day18"></a>

# Day 18

<a class="anchor" id="day19"></a>

# Day 19

<a class="anchor" id="day20"></a>

# Day 20

<a class="anchor" id="day21"></a>

# Day 21

<a class="anchor" id="day22"></a>

# Day 22

<a class="anchor" id="day23"></a>

# Day 23

<a class="anchor" id="day24"></a>

# Day 24

<a class="anchor" id="day25"></a>

# Day 25